1. Importamos el modelo que creamos

In [1]:
from tensorflow.keras.models import load_model

# Cargar el modelo
model = load_model('mnist_model.h5')
print("Modelo cargado correctamente.")


✅ Modelo cargado correctamente.


Configuramos para abrir la cámara y que prediga en tiempo real.

In [5]:
%%writefile real_time_prediction.py
import cv2
import numpy as np
import tensorflow as tf

# Cargar el modelo
model = tf.keras.models.load_model("mnist_model.h5")  # cambia el nombre si es distinto
print("Modelo cargado correctamente.")
print("Presiona 'q' para salir")

# Abrir la cámara
cap = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Invertir y umbralizar la imagen (ajusta el valor 128 si se necesita)
        _, threshed = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

        # Redimensionar a 28x28 píxeles y normalizar
        resized = cv2.resize(threshed, (28, 28))
        input_img = resized.reshape(1, 28 * 28) / 255.0  # <- plano porque tu modelo usa (784,)

        # Predicción
        prediction = model.predict(input_img, verbose=0)
        predicted_label = np.argmax(prediction)

        # Mostrar la predicción sobre el video
        frame_with_prediction = cv2.putText(
            frame.copy(),
            f"Prediccion: {predicted_label}",
            (10, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0),
            2,
            cv2.LINE_AA
        )

        # Mostrar ventanas
        cv2.imshow('Predicción en tiempo real', frame_with_prediction)
        cv2.imshow('Entrada (28x28)', cv2.resize(resized, (280, 280), interpolation=cv2.INTER_NEAREST))

        # Presiona 'q' para salir
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("\n Interrumpido manualmente.")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Cámara liberada correctamente.")


Overwriting real_time_prediction.py


In [8]:
!python real_time_prediction.py


Modelo cargado correctamente.
Presiona 'q' para salir
Cámara liberada correctamente.


2025-10-31 21:55:12.519150: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-31 21:55:14.461738: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-31 21:55:16.031705: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
